In [1]:
cd("/home/rafael/UNAM/Sanders/Mancuerna/Dumbbell/Dumbbell-Julia/Codigo-julia")

include("dumbbell_type.jl")
include("Gram_Schmidt.jl")
include("dumbbell_lyapunov_functions.jl")


evol_δΓ (generic function with 3 methods)

In [2]:
walls=reshape([[-3, 3], [-2, 2]], 2, 2)

2x2 Array{Int32,2}:
 -3  -2
  3   2

In [3]:
hw = walls[1:2]
vw = walls[3:4]

hw, vw

([-3,3],[-2,2])

In [16]:
num, lets = ([1,2,4], [5,6,7])
num

3-element Array{Int32,1}:
 1
 2
 4

In [21]:
?error

Base.error(message::String)

   Raise an error with the given message


In [26]:
function check_dumbbell_test(check_frequency, db::dumbbell, walls, exponents)
    cc = db.collision_counter
    cf = check_frequency
    if cc%cf==0
        exp_time, exp_cc = exponents
        sum1 = 0.; sum2 = 0.
        for j in 1:6
            sum1 += exp_time[cc,j]
            sum2 += exp_cc[cc,j]
        end
        
        println("Sum of Lyapunov Exponentes after \t"*string(cc)*" collisions")
        println(sum1, "\t", sum2)
        
        x1, y1 = particle_position(db, 1)[1:2];   x2, y2 = particle_position(db, 2)[1:2]
        vert_walls = walls[1:2]; horiz_walls = walls[3:4] # coordinates of the vertical and horizontal walls, respectively 
        err = 1e-10
        
        if ~(minimum(vert_walls)-err< x1 < maximum(vert_walls)+err) || 
            ~(minimum(vert_walls)-err< x2 < maximum(vert_walls)+err) ||
            ~(minimum(horiz_walls)-err< y1 < maximum(horiz_walls)+err) || 
            ~(minimum(horiz_walls)-err< y2 < maximum(horiz_walls)+err)
            
            println("Dumbbell position\n(particle 1; particle 2)")
            println(x1,"\t", y1,";\t", x2,"\t", y2)
            
            error("The dumbbell is out of the billiard")
            
        end
        
    end
    
end
        
    

check_dumbbell_test (generic function with 1 method)

In [27]:
function disp_vecs_transformation(db::dumbbell, walls, norm_disp_vecs)
    
    ### Given a state of the dumbbell, 'db', inside thw billiard with frontiers, 'walls,
    ### And a set of normalized displacement vectors, 'norm_disp_vecs',
    ### This function will calculate the evolution of the displacement vectors (DV); their transformation
    ### during the collision, and a new set of orthogonal and orthonormal vectors: 'ortho_disp_vecs'; 
    ### norm_disp_vecs2.
    ### The output is a tuple of the collision time ('tc') it took the dumbbell to reach for the next wall,
    ### ortho_disp_vecs, and norm_disp_vecs2
    
    vert_walls = walls[1:2]; horiz_walls = walls[3:4] # coordinates of the vertical and horizontal walls, respectively 

    Γ0,  Γf, wall, part, ct = collision(db, vert_walls, horiz_walls)
    
    disp_vecs0 = Array{Float64}[]
    for v in norm_disp_vecs
        vf = evol_δΓ(v, ct, l0, m0) # evolution of each of the initial normalized DV
        push!(disp_vecs0, [vf])
    end

    disp_vecs_F = Array{Float64}[]
    for dΓ in disp_vecs0
        dΓp = displacement_vector_collision_map(Γ0, dΓ, part, wall, l0, m0)
        push!(disp_vecs_F, [dΓp])
    end
    
    ortho_disp_vecs, norm_disp_vecs2 = Gram_Schmidt(disp_vecs_F)
    
    ct, ortho_disp_vecs, norm_disp_vecs2
end

function lyapunov_spectrum(db::dumbbell, walls, N, check=false, check_times=3)
    # N=number of collisions
    srand(5) #seed for random number generator  
    
    # in case 'check' variable is True, every 'check_frequency' collisions,
    # the sum of the exponents will be computed and a test will be run to guarantee that the dumbbell
    # is within the frontiers of the billiard (+/- the error with which a collision is determined)
    check_frequency = int(N/check_times)  
    
    rand_vecs = ([rand(6) for i in 1:6]) # initial random displacement vectors
    ortho_disp_vecs, norm_disp_vecs = Gram_Schmidt(rand_vecs)  # initial DV, orthogonalized and orthonormalized
    
    col_count = 0 # collision counter

    λ_col = zeros(Float64,N,6); λ_time = zeros(Float64,N,6) #value of the LEs
    χs = zeros(Float64,N,6) # log of the norm of (the sum of) the DV
    
    # arrays of the collision time and the total elapsed time (until each collision)
    col_times = Float64[]; elapsed_time = Float64[]; 
    t_T= 0. # initial time
    
    # main loop for calculating the LEs for the N collisions
    for k in 1:N
        col_count += 1
        
        #collision time, orthogonal vectors, orthonormal vectors after the k-th collision
        ct, O_vecs, ON_vecs = disp_vecs_transformation(db, walls, norm_disp_vecs)
        
        t_T += ct # add the current collision time to the elapsed time
        push!(elapsed_time, t_T);  push!(col_times, ct) # register the times
        
        for (j,ov) in enumerate(O_vecs)
            if k>1
                χs[k, j] = χs[k-1, j] + log(norm(ov))
            else
                χs[1,j] = log(norm(ov))
            end
            
            # LEs obtained with the number of collisions and the elapsed time 
            λ_col[k,j] = χs[k,j]/(k); λ_time[k,j] = χs[k,j]/t_T 
        end
        
        if check
            check_dumbbell_test(check_frequency, db, walls, (λ_col, λ_time))
        end
            
        
        norm_disp_vecs = ON_vecs # set the new DV as the normalized DV after the collision
        
    end
    
    λ_col, λ_time, elapsed_time, col_times
end

lyapunov_spectrum (generic function with 3 methods)

In [30]:
rcm = [1., 1.]; l0=0.2; m0=1.; vcm = [2.1, 0.6]; θ=π/6; ω=2.

db = dumbbell(rcm, θ, vcm, ω, m0, l0, 0)

walls = [[-3,3], [-2, 2]]
nc=100

lyapunov_spectrum(db, walls, nc, true, 10)

Sum of Lyapunov Exponentes after 	10 collisions
1.9317880628477724e-14	1.6209256159527285e-14
Sum of Lyapunov Exponentes after 	20 collisions
1.0880185641326534e-14	8.659739592076221e-15
Sum of Lyapunov Exponentes after 	30 collisions
-2.2670754162845697e-13	-1.7186252421197423e-13
Sum of Lyapunov Exponentes after 	40 collisions
-1.7275070263167436e-13	-1.2856382625159313e-13
Sum of Lyapunov Exponentes after 	50 collisions
-1.3078427230084344e-13	-9.459100169806334e-14
Sum of Lyapunov Exponentes after 	60 collisions
-1.092459456231154e-13	-8.393286066166183e-14
Sum of Lyapunov Exponentes after 	70 collisions
-8.593126210598712e-14	-5.417888360170764e-14
Sum of Lyapunov Exponentes after 	80 collisions
-7.260858581048524e-14	-4.463096558993129e-14
Sum of Lyapunov Exponentes after 	90 collisions
-5.950795411990839e-14	-3.708144902248023e-14
Sum of Lyapunov Exponentes after 	100 collisions
-4.063416270128073e-14	-2.55351295663786e-14


(
100x6 Array{Float64,2}:
 2.25869  1.85146   0.389209   -0.778905   -0.249198  -3.47125
 1.62169  1.159     0.554522   -0.38981    -0.913962  -2.03144
 2.22333  1.57687   0.566597   -0.310255   -1.84241   -2.21413
 1.52041  0.783152  0.425829   -0.172616   -1.1008    -1.45598
 1.96325  0.977456  0.421425   -0.257026   -1.22746   -1.87764
 1.75712  1.0264    0.6871     -0.237708   -1.04723   -2.18568
 1.92084  1.16304   0.264365   -0.166461   -1.07624   -2.10554
 1.91366  1.04765   0.0873642  -0.0589306  -0.779537  -2.21021
 1.54265  0.959909  0.120417   -0.0491905  -0.601016  -1.97277
 1.48107  0.835131  0.118432    0.098121   -0.682031  -1.85072
 1.38484  0.789874  0.107156    0.136821   -0.656163  -1.76253
 1.49967  0.825813  0.155631   -0.185882   -0.58041   -1.71482
 1.3852   0.790973  0.146869   -0.199938   -0.562658  -1.56044
 ⋮                                                     ⋮      
 1.73567  0.95439   0.448988   -0.384219   -0.899433  -1.85539
 1.71492  0.933169  0.442349 

In [11]:
srand(5)